## Рекомендательные системы




#### Словарь с предпочтениями пользователи-фильмы

In [1]:
# Словарь кинокритиков и выставленных ими оценок для небольшого набора
# данных о фильмах
critics = {'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
                         'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
                         'The Night Listener': 3.0},
           'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
                            'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 3.5},
           'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
                                'Superman Returns': 3.5, 'The Night Listener': 4.0},
           'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
                            'The Night Listener': 4.5, 'Superman Returns': 4.0,
                            'You, Me and Dupree': 2.5},
           'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                            'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 2.0},
           'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                             'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
           'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}}

In [2]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

In [3]:
critics['Lisa Rose']['Lady in the Water']

2.5

In [4]:
# вычисление расстояния Евклида
from math import sqrt

print(sqrt(pow(5 - 4, 2) + pow(4 - 1, 2)))

3.1622776601683795


In [5]:
# вычисление сходства
1 / (1 + sqrt(pow(5 - 4.5, 2) + pow(5 - 5, 2)))

0.6666666666666666

In [6]:
from numpy import exp

exp(-0.3 * sqrt(pow(5 - 4.5, 2) + pow(5 - 5, 2)))

0.8607079764250578

### Задание 1
По аналогии реализуйте функции для расчета расстояния между двумя пользователями: __sim_distance__(расстояние Еквликда) и __sim_kernel__. Если общих оценок нет - возвращаем __0__.

In [54]:
def sim_distance(prefs, person1, person2):
    """
    Возвращает сходство person1 и person2 на основе расстояния    
    """
    common_films = set(prefs[person1].keys()).intersection(set(prefs[person2].keys()))
    sum_ = [(prefs[person1][film] - prefs[person2][film]) ** 2 for film in common_films]
    
    return 1 / (1 + sum(sum_))

In [55]:
sim_distance(critics, 'Lisa Rose','Toby')

0.2222222222222222

In [56]:
def sim_kernel(prefs, person1, person2, alpha=0.3):
    """
    Возвращает сходство person1 и person2 
    :param alpha: 
    """
    # Ваш код здесь
    common_films = set(prefs[person1].keys()).intersection(set(prefs[person2].keys()))
    sum_ = [(prefs[person1][film] - prefs[person2][film]) ** 2 for film in common_films]
    
    # exp(-0.3 * sqrt(pow(5 - 4.5, 2) + pow(5 - 5, 2)))
    return exp(-alpha * sum(sum_))

In [57]:
sim_kernel(critics, 'Lisa Rose','Toby')

0.3499377491111553

In [58]:
def sim_pearson(prefs, p1, p2):
    """
    # Возвращает коэффициент корреляции Пирсона между p1 и p2    
    :param prefs: 
    :param p1: 
    :param p2: 
    :return: 
    """
    # Получить список предметов, оцененных обоими
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item] = 1

    # Если нет ни одной общей оценки, вернуть 0
    if len(si) == 0: return 0

    # Количество соместно оцененных фильм
    n = len(si)

    # Вычислить сумму всех предпочтений
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])

    # Вычислить сумму квадратов
    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])

    # Вычислить сумму произведений
    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])

    # Вычислить коэффициент Пирсона
    num = pSum - (sum1 * sum2 / n)
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0: return 0

    r = num / den

    return r

In [59]:
sim_pearson(critics,'Lisa Rose','Gene Seymour'), sim_distance(critics, 'Lisa Rose','Gene Seymour')

(0.39605901719066977, 0.14814814814814814)

## Ранжирование критиков
### Задание 2
Реализуйте функцию __top_mathes__ для поиска наилучших соответствий для конкретного человека


In [60]:
def top_matches(prefs,person,n=5,similarity=sim_pearson):
    """
    Возвращает список наилучших соответствий для человека из словаря prefs.
    :param n: количество результатов
    :param similarity: функция подобия
    :return list(<similarity, name>)
    """
    # Ваш код здесь
    result = sorted([(similarity(prefs, person, other), other) 
                     for other in prefs if other != person], reverse=True)
        
    return result[:n]

In [61]:
top_matches(critics,'Toby',n=3, similarity=sim_distance)

[(0.3076923076923077, 'Mick LaSalle'),
 (0.2857142857142857, 'Michael Phillips'),
 (0.23529411764705882, 'Claudia Puig')]

In [62]:
top_matches(critics,'Toby',n=5, similarity=sim_kernel)

[(0.5091564206075492, 'Mick LaSalle'),
 (0.4723665527410147, 'Michael Phillips'),
 (0.37719235356315695, 'Claudia Puig'),
 (0.3499377491111553, 'Lisa Rose'),
 (0.10539922456186433, 'Jack Matthews')]

## Рекомендация фильмов (User-based подход)

In [106]:
def get_recommendations(prefs, person, similarity=sim_pearson):
    """
    Получить рекомендации для заданного человека, пользуясь взвешенным средним
    оценок, данных всеми остальными пользователями
    """
    totals = {}
    simSums = {}
    for other in prefs:
        # сравнивать меня с собой же не нужно
        if other == person: continue
        sim = similarity(prefs, person, other)
        # игнорировать нулевые и отрицательные оценки
        if sim <= 0: continue
        for item in prefs[other]:
            # оценивать только фильмы, которые я еще не смотрел
            if item not in prefs[person] or prefs[person][item] == 0:
                # Коэффициент подобия * Оценка
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item] * sim
                # Сумма коэффициентов подобия
                simSums.setdefault(item, 0)
                simSums[item] += sim
    # Создать нормированный список
    rankings = [(total / simSums[item], item) for item, total in totals.items()]
    # Вернуть отсортированный список
    rankings.sort()
    rankings.reverse()
    return rankings

In [64]:
get_recommendations(critics,'Toby')


[(3.3477895267131013, 'The Night Listener'),
 (2.832549918264162, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]

In [65]:
get_recommendations(critics,'Toby', sim_kernel)

[(3.546911137052965, 'The Night Listener'),
 (2.729687080418633, 'Lady in the Water'),
 (2.518799545549861, 'Just My Luck')]

## Сходство предметов
### Задание 3 
Написать функцию __transoform_prefs__ для замены :

{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5}}

на:

{'Lady in the Water':{'Lisa Rose':2.5,'Gene Seymour':3.0},
'Snakes on a Plane':{'Lisa Rose':3.5,'Gene Seymour':3.5}}

In [105]:
def transform_prefs(prefs):
    result={}
    # Ваш код здесь
    for person in prefs:
      for film in prefs[person]:
        result.setdefault(film, {})
        result[film][person] = prefs[person][film]
    return result

In [67]:
movies = transform_prefs(critics)

In [68]:
movies

{'Just My Luck': {'Claudia Puig': 3.0,
  'Gene Seymour': 1.5,
  'Lisa Rose': 3.0,
  'Mick LaSalle': 2.0},
 'Lady in the Water': {'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 2.5,
  'Michael Phillips': 2.5,
  'Mick LaSalle': 3.0},
 'Snakes on a Plane': {'Claudia Puig': 3.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 4.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.0,
  'Mick LaSalle': 4.0,
  'Toby': 4.5},
 'Superman Returns': {'Claudia Puig': 4.0,
  'Gene Seymour': 5.0,
  'Jack Matthews': 5.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.5,
  'Mick LaSalle': 3.0,
  'Toby': 4.0},
 'The Night Listener': {'Claudia Puig': 4.5,
  'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 3.0,
  'Michael Phillips': 4.0,
  'Mick LaSalle': 3.0},
 'You, Me and Dupree': {'Claudia Puig': 2.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 3.5,
  'Lisa Rose': 2.5,
  'Mick LaSalle': 2.0,
  'Toby': 1.0}}

In [69]:
top_matches(movies,'Snakes on a Plane', 5, sim_pearson)

[(0.7637626158259785, 'Lady in the Water'),
 (0.11180339887498941, 'Superman Returns'),
 (-0.3333333333333333, 'Just My Luck'),
 (-0.5663521139548527, 'The Night Listener'),
 (-0.6454972243679047, 'You, Me and Dupree')]

In [70]:
get_recommendations(movies,'Lady in the Water', sim_distance)

[(3.352635310801229, 'Claudia Puig'), (2.4730878186968837, 'Toby')]

## Коллаборативная фильтрация по сходству объектов (Item-based collaborative filtering)

<img src='https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif' style="width: 300px">

In [104]:
def calculate_similar_items(prefs, n=10):
    # Создать словарь, содержащий для каждого образца те образцы, которые
    # больше всего похожи на него.
    result = {}
    # Обратить матрицу предпочтений, чтобы строки соответствовали образцам
    item_prefs = transform_prefs(prefs)
    c = 0
    for item in item_prefs:
        # Обновление состояния для больших наборов данных
        c += 1
        if c % 1000 == 0:
            print("%d / %d" % (c, len(item_prefs)))
        # Найти образцы, максимально похожие на данный
        scores = top_matches(item_prefs, item, n=n, similarity=sim_pearson)
        result[item] = scores
    return result


In [84]:
itemsim=calculate_similar_items(critics)
itemsim

{'Just My Luck': [(0.5555555555555556, 'The Night Listener'),
  (-0.3333333333333333, 'Snakes on a Plane'),
  (-0.42289003161103106, 'Superman Returns'),
  (-0.4856618642571827, 'You, Me and Dupree'),
  (-0.9449111825230676, 'Lady in the Water')],
 'Lady in the Water': [(0.7637626158259785, 'Snakes on a Plane'),
  (0.4879500364742689, 'Superman Returns'),
  (0.3333333333333333, 'You, Me and Dupree'),
  (-0.6123724356957927, 'The Night Listener'),
  (-0.9449111825230676, 'Just My Luck')],
 'Snakes on a Plane': [(0.7637626158259785, 'Lady in the Water'),
  (0.11180339887498941, 'Superman Returns'),
  (-0.3333333333333333, 'Just My Luck'),
  (-0.5663521139548527, 'The Night Listener'),
  (-0.6454972243679047, 'You, Me and Dupree')],
 'Superman Returns': [(0.6579516949597695, 'You, Me and Dupree'),
  (0.4879500364742689, 'Lady in the Water'),
  (0.11180339887498941, 'Snakes on a Plane'),
  (-0.1798471947990544, 'The Night Listener'),
  (-0.42289003161103106, 'Just My Luck')],
 'The Night L

In [103]:
def get_recommended_items(prefs, itemMatch, user):
    userRatings = prefs[user]
    scores = {}
    totalSim = {}

    # Цикл по образцам, оцененным данным пользователем
    for (item, rating) in userRatings.items():
        # Цикл по образцам, похожим на данный
        for (similarity, item2) in itemMatch[item]:
            # Пропускаем, если пользователь уже оценивал данный образец
            if item2 in userRatings: continue
            # Взвешенная суммы оценок, умноженных на коэффициент подобия
            scores.setdefault(item2, 0)
            scores[item2] += similarity * rating
            # Сумма всех коэффициентов подобия
            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity
            if totalSim[item2] == 0: totalSim[item2] = 0.0000001  # чтобы избежать деления на ноль
    # Делим каждую итоговую оценку на взвешенную сумму, чтобы вычислить
    # среднее
    rankings = [(score / totalSim[item], item) for item, score in scores.items()]

    # Возвращает список rankings, отсортированный по убыванию
    rankings.sort()
    rankings.reverse()
    return rankings

In [86]:
get_recommended_items(critics,itemsim,'Toby')

[(3.610031066802182, 'Lady in the Water'), (3.531395034185976, 'The Night Listener'), (2.9609998607242685, 'Just My Luck')]


[(3.610031066802182, 'Lady in the Water'),
 (3.531395034185976, 'The Night Listener'),
 (2.9609998607242685, 'Just My Luck')]

## Рекомендация на данных MovieLens

Источник: http://grouplens.org/datasets/movielens/

### Задание 4 
Загрузить данные по ссылке: http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Реализовать функцию __load_movies__, которая создает словарь, аналогичный словарю __critics__. 

In [87]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -N
!unzip ml-latest-small.zip
!ls

--2019-06-06 16:46:40--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ml-latest-small.zip’ not modified on server. Omitting download.

Archive:  ml-latest-small.zip
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  
links.csv	     ratings.csv
main.py		     README.txt
ml-latest-small      recommendationSystems_done.ipynb
ml-latest-small.zip  recommendationSystems.ipynb
movies.csv	     tags.csv


In [88]:
!mv ml-latest-small/* . 
!ls

links.csv	     ratings.csv
main.py		     README.txt
ml-latest-small      recommendationSystems_done.ipynb
ml-latest-small.zip  recommendationSystems.ipynb
movies.csv	     tags.csv


In [89]:
!head -n 5 movies.csv
!head -n 5 ratings.csv

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
userId,movieId,rating,timestamp
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815


In [102]:
import csv
def load_movies():
    movies = {}
    with open("movies.csv", "r") as f:
      csv_reader = csv.reader(f, delimiter=',', quotechar='"')
      for index, key in enumerate(csv_reader):
        if index == 0:
          continue
        movie_id, title, genres = key
        movies[movie_id] = title
    
    prefs = {}
    with open("ratings.csv", "r") as f:
      csv_reader = csv.reader(f, delimiter=',', quotechar='"')
      for index, key in enumerate(csv_reader):
        if index == 0:
          continue
        user_id, movie_id, rating, timestamp = key
        prefs.setdefault(user_id, {})
        prefs[user_id][movies[movie_id]] = float(rating)
        
    return prefs

In [91]:
prefs=load_movies()
print(len(prefs))
prefs['87']

610


{"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)": 5.0,
 'Battle Royale (Batoru rowaiaru) (2000)': 3.5,
 'Battlefield Earth (2000)': 2.0,
 "Bill & Ted's Excellent Adventure (1989)": 3.5,
 'Carrie (1976)': 3.5,
 'Contact (1997)': 4.0,
 'Dancer in the Dark (2000)': 4.0,
 'Family Man, The (2000)': 3.5,
 'Indian in the Cupboard, The (1995)': 3.0,
 'Little Princess, A (1995)': 4.0,
 'Monty Python and the Holy Grail (1975)': 5.0,
 'My Neighbor Totoro (Tonari no Totoro) (1988)': 5.0,
 'Mystery Science Theater 3000: The Movie (1996)': 4.5,
 'Network (1976)': 4.5,
 'Producers, The (1968)': 4.5,
 'Secret Garden, The (1993)': 4.0,
 'Secret of NIMH, The (1982)': 3.0,
 'Sleeper (1973)': 4.0,
 'Soylent Green (1973)': 4.0,
 'Spirited Away (Sen to Chihiro no kamikakushi) (2001)': 5.0,
 'Tank Girl (1995)': 3.5}

In [92]:
len(prefs['87'])

21

In [113]:
get_recommendations(prefs,'87')[0:30]

[(5.0, 'Zeitgeist: Moving Forward (2011)'),
 (5.0, 'Wow! A Talking Fish! (1983)'),
 (5.0, 'Wonder Woman (2009)'),
 (5.0, 'Woman Under the Influence, A (1974)'),
 (5.0, 'Woman Is a Woman, A (femme est une femme, Une) (1961)'),
 (5.0, 'Winter in Prostokvashino (1984)'),
 (5.0, 'Winnie the Pooh and the Day of Concern (1972)'),
 (5.0, 'Winnie the Pooh Goes Visiting (1971)'),
 (5.0, 'Winnie Pooh (1969)'),
 (5.0, 'Wings, Legs and Tails (1986)'),
 (5.0, 'When Worlds Collide (1951)'),
 (5.0, 'What Men Talk About (2010)'),
 (5.0, 'Watermark (2014)'),
 (5.0, 'War for the Planet of the Apes (2017)'),
 (5.0, 'Vovka in the Kingdom of Far Far Away (1965)'),
 (5.0, 'Very Potter Sequel, A (2010)'),
 (5.0, 'Vampire in Venice (Nosferatu a Venezia) (Nosferatu in Venice) (1986)'),
 (5.0, 'Vagabond (Sans toit ni loi) (1985)'),
 (5.0, 'Vacations in Prostokvashino (1980)'),
 (5.0, 'Unfaithfully Yours (1948)'),
 (5.0, "Tyler Perry's I Can Do Bad All by Myself (2009)"),
 (5.0, 'Two Family House (2000)'),
 (5.0

In [94]:
itemsim=calculate_similar_items(prefs,n=10)

1000 / 9719
2000 / 9719
3000 / 9719
4000 / 9719
5000 / 9719
6000 / 9719
7000 / 9719
8000 / 9719
9000 / 9719


In [112]:
print(itemsim["What's Eating Gilbert Grape (1993)"])

[(1.000000000000016, 'Year of Living Dangerously, The (1982)'), (1.000000000000016, 'Vera Drake (2004)'), (1.000000000000016, 'Namesake, The (2006)'), (1.000000000000016, 'Ice Age: Dawn of the Dinosaurs (2009)'), (1.000000000000016, 'Better Luck Tomorrow (2002)'), (1.0000000000000133, 'Tenacious D in The Pick of Destiny (2006)'), (1.0000000000000133, 'Love Song for Bobby Long, A (2004)'), (1.0000000000000089, 'Everything You Always Wanted to Know About Sex * But Were Afraid to Ask (1972)'), (1.000000000000007, 'Straw Dogs (1971)'), (1.000000000000007, 'Stepmom (1998)')]


In [111]:
for result in get_recommended_items(prefs,itemsim,'87')[:40]:
    print(result)


(5.0, 'You Can Count on Me (2000)')
(5.0, 'Year One (2009)')
(5.0, 'This Is the End (2013)')
(5.0, 'Them! (1954)')
(5.0, 'Tears of the Sun (2003)')
(5.0, 'Stepmom (1998)')
(5.0, 'Spider (2002)')
(5.0, 'Shut Up & Sing (2006)')
(5.0, 'Session 9 (2001)')
(5.0, 'Rambo (Rambo 4) (2008)')
(5.0, 'Percy Jackson & the Olympians: The Lightning Thief (2010)')
(5.0, 'Penguins of Madagascar (2014)')
(5.0, 'Other Guys, The (2010)')
(5.0, 'Odd Couple, The (1968)')
(5.0, 'Nebraska (2013)')
(5.0, 'Namesake, The (2006)')
(5.0, 'Mystery, Alaska (1999)')
(5.0, 'Monuments Men, The (2014)')
(5.0, 'Million Dollar Hotel, The (2001)')
(5.0, 'Midnight Special (2015)')
(5.0, 'Marathon Man (1976)')
(5.0, 'Mad Hot Ballroom (2005)')
(5.0, "M. Hulot’s Holiday (Mr. Hulot's Holiday) (Vacances de Monsieur Hulot, Les) (1953)")
(5.0, 'Love and Other Drugs (2010)')
(5.0, 'Looking for Richard (1996)')
(5.0, 'Layer Cake (2004)')
(5.0, 'Howards End (1992)')
(5.0, 'Girl on the Bridge, The (Fille sur le pont, La) (1999)')
(5.0